# setup

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import numpy as np
import time

from IPython.display import display

plt.style.use('fivethirtyeight')
%config InlineBackend.figure_format = 'retina'

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo utilizado: {device}")

Dispositivo utilizado: cuda:0


# functions

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, conv_layers=None, fc_layers=None, dropout_probs=None):
        """
        `todo`
        """
        super(CNNModel, self).__init__()
        #
        if conv_layers is None:
            conv_layers = [(32, 3, 1, 1), (64, 3, 1, 1)]  # (filtros, kernel_size, stride, padding)
        if fc_layers is None:
            fc_layers = [512, 10]
        if dropout_probs is None:
            dropout_probs = [0.0] * (len(fc_layers) - 1)
        # 
        self.conv_layers = nn.ModuleList()
        in_channels = 3
        for i, (filters, kernel_size, stride, padding) in enumerate(conv_layers):
            # convolution
            self.conv_layers.append(nn.Conv2d(in_channels, filters, kernel_size, stride, padding))
            # activation
            self.conv_layers.append(nn.ReLU())
            # pooling
            self.conv_layers.append(nn.MaxPool2d(2, 2))
            in_channels = filters
        #
        x = torch.randn(1, 3, 32, 32)  # CIFAR-10
        for layer in self.conv_layers:
            x = layer(x)
        conv_output_size = x.view(1, -1).size(1)
        # mlp
        self.fc_layers = nn.ModuleList()
        in_features = conv_output_size
        for i in range(len(fc_layers)):
            out_features = fc_layers[i]
            self.fc_layers.append(nn.Linear(in_features, out_features))
            # add relu e dropout
            if i < len(fc_layers) - 1:
                self.fc_layers.append(nn.ReLU())
                if dropout_probs[i] > 0:
                    self.fc_layers.append(nn.Dropout(dropout_probs[i]))
            in_features = out_features
    
    def forward(self, x):
        # conv
        for layer in self.conv_layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        # mlp
        for layer in self.fc_layers:
            x = layer(x)
        return x

In [ ]:
def load_data(batch_size=64, use_data_augmentation=False, validation_split=0.1):
    """
    `todo`
    """
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    #
    if use_data_augmentation:
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
    else:
        transform_train = transform_test
    #
    train_dataset = torchvision.datasets.CIFAR10(
        root='/home/dusoudeth/Documentos/github/ppgcc-neural-networks-homeworks/data/atividade_04',
        train=True,
        download=True,
        transform=transform_train
    )
    val_size = int(len(train_dataset) * validation_split)
    train_size = len(train_dataset) - val_size
    train_set, val_set = random_split(train_dataset, [train_size, val_size])
    #
    test_dataset = torchvision.datasets.CIFAR10(
        root='/home/dusoudeth/Documentos/github/ppgcc-neural-networks-homeworks/data/atividade_04',
        train=False, 
        download=True, 
        transform=transform_test
    )
    #
    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    #
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    print(f"Tamanho do treino: {train_size} imagens")
    print(f"Tamanho da validação: {val_size} imagens")
    print(f"Tamanho do teste: {len(test_dataset)} imagens")
    return train_loader, val_loader, test_loader, classes

## analysis

In [7]:
i = 2
train = pd.read_csv(f"../data/atividade_01/train_dataset{i}.csv")
test = pd.read_csv(f"../data/atividade_01/test_dataset{i}.csv")
display(train.head())
display(test.head())

X_train = train.iloc[:, :-1].values
y_train = train.iloc[:, -1].values
X_test = test.iloc[:, :-1].values
y_test = test.iloc[:, -1].values

task = 'binary'
input_features = X_train.shape[1]
output_size = get_output_size(y_train, task=task)

train_loader, val_loader, test_loader = prepare_classification_data(
    X_train, 
    y_train, 
    X_test = X_test, 
    y_test = y_test,
    batch_size = 32,
    task = task,
    convert_labels = True
)

,x1,x2,label
0,-0.882159,-0.511697,-1
1,0.056971,0.101538,1
2,0.987759,-0.000616,-1
3,1.108407,0.017528,-1
4,-0.555163,0.770001,-1


,x1,x2,label
0,-0.141852,0.102203,1
1,0.134028,-1.027246,-1
2,0.171773,-0.896942,-1
3,0.071050,-0.193340,1
4,-0.829970,0.547396,-1
